In [5]:
%matplotlib inline
import IPython
import pylab

import matplotlib.pyplot as plt
import numpy as np
import pylab as py
import scipy as sp

import scipy.signal

from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from elasticsearch_dsl.connections import connections
from datetime import datetime

connections.create_connection(timeout=180)
pylab.rcParams['figure.figsize'] = (16, 4)
plt.hold(True)
plt.close()

In [2]:
def parse_iso_date(str):
    return datetime.strptime(str, "%Y-%m-%dT%H:%M:%S.%fZ")

In [3]:
def hist_data(terms=None, start='1900-01-01', end='2020-01-01'):
    q = Search(index='records-hep')[0:0]
    if terms:
        q = q.query('match', abstract=terms)
    q = q.filter('range', earliest_date={'gte': start, 'lt': end})
    q.aggs.bucket(
        'group_by_date', 
        'date_histogram', 
        field='earliest_date', 
        interval='day', 
        format='date_optional_time'
    )
    return q.execute().aggregations.group_by_date.buckets

In [4]:
def unzip(data):
    x, y = zip(*[(parse_iso_date(e.key_as_string), e.doc_count) for e in data])
    return np.array(x), np.array(y)